# Clasificación de comentarios
Víctor Daniel Cruz González

# Objetivo
Establecer un ambiente de comunicación seguro y de calidad para los usuarios


## Contexto

Para este proyecto, requiero de tener una plataforma web capaz de indicar qué tan bueno o malo es un comentario, basándome en un modelo de Recurrent Neuronal Network que detallaré más adelante.

Esta plataforma es un sitio de recetas en el que los usuarios pueden compartir videos y sus recetarios. Adicionalmente, otros pueden comentar cualquier cosa sobre esta. Sin embargo, se prohibe el uso de lenguaje ofensivo al momento de comentar. Por ende, la red neuronal sirve para detectar estos casos y permitir o no mandar el mensaje.

# Antecedentes

## Recurrent Neuronal Network
Una red neuronal recurrente (RNN) es un tipo de red neuronal que utiliza datos secuenciales o series de tiempo. Estos algoritmos de aprendizaje profundo se utilizan para problemas temporales, como la traducción de idiomas, el procesamiento del lenguaje natural (nlp) o el reconocimiento de voz.

![Unrolled network](unrolled-network.png)

Mientras que las redes neuronales profundas asumen que las entradas y salidas son independientes entre sí, la salida de las recurrentes depende de los elementos anteriores. Asimismo, comparten parámetros en cada capa de la red. Las redes neuronales recurrentes comparten el mismo parámetro de peso dentro de cada capa de la red. Estos pesos todavía se ajustan en los procesos de retropropagación y descenso de gradiente para facilitar el aprendizaje por refuerzo.

Las redes neuronales recurrentes aprovechan el algoritmo de retropropagación a través del tiempo (BPTT) para determinar los gradientes, pues es específico de los datos de secuencia. Este algoritmo implica que el modelo se entrena a sí mismo calculando errores desde su capa de salida hasta su capa de entrada. Estos cálculos nos permiten ajustar los parámetros del modelo. BPTT se diferencia del enfoque tradicional en que BPTT suma los errores en cada paso de tiempo, mientras que las redes de retroalimentación no necesitan sumar errores, ya que no comparten parámetros en cada capa.

![BPPT](bptt.svg)

A través de este proceso, los RNN tienen dos retos, conocidos como gradientes explosivos y gradientes que desaparecen. Estos problemas se definen por el tamaño del gradiente. Cuando el gradiente es demasiado pequeño, continúa haciéndolo más pequeño, actualizando los parámetros de peso hasta que se vuelven 0. Cuando eso ocurre, el algoritmo ya no está aprendiendo. Los gradientes explosivos se producen cuando el xambio de valor es demasiado grande, creando un modelo inestable. En este caso, los pesos del modelo crecerán demasiado, hasta números que sobrepasan las librerías de las computadoras. Una solución a estos problemas es reducir la cantidad de capas ocultas dentro de la red neuronal, eliminando parte de la complejidad del modelo RNN.

## Arquitectura

Dado que es una plataforma web, en este projecto use Node.js y Python para establecer la comunicación y la RNN. Específicamente, Tensorflow ayudo a esto y Flask a establecer comunicación entre Node.js y el modelo. Para hacerlo en un ambiente controlado, elegí Docker y Docker Compose.

# Implementación

Inicializando librerias.

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
def plotGraphs(history, metric):
    """
    Display graph of history and metric
    """
    plt.plot(history.history[metric])
    plt.plot(history.history['val_' + metric], '')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val' + metric])

Obteniendo dataset de IMDB

In [3]:
dataset, info = tfds.load(
    'imdb_reviews', with_info=True, as_supervised=True)
trainDataset, testDataset = dataset['train'], dataset['test']
print(trainDataset.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


Generando ejemplo

In [4]:
for example, label in trainDataset.take(1):
    print('text: ', example.numpy())
    print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


Mezclando los datos

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

trainDataset = trainDataset.shuffle(BUFFER_SIZE).batch(
    BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in trainDataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'had some lovely poetic bits but is really just an artsy-fartsy toss-together with no direction or resolution. how do these people get through film school? who gives them money to make this crap? could have been so much more, fine lead actor, and i always like Fairuza Balk, but come on, the alt-rock metaphor of just staring vacantly unable to find anything compelling is just so tired, and it sure doesn\'t make for good films. the director needs to go away and live life for a good long while and not come back to the camera until they really have something to say. this is like the throw-spaghetti-at-the-wall school of art-making, just juxtapose a bunch of earnest imagery and hope hope hope like hell that poetry emerges. that can work, if the director actually has any kind of vision, or has a brain that knows when it\'s in the presence of potential, but here it\'s just space filler, of no consequence. i felt the lazy ending coming moments before it hit, and was yelling "you lazy

Creando text encoder

Esto implica que el TextVectorization cambia y etique las palabras para que después sean usadas para el modelo. Hay que notar que los UNK se generan cuando el tamaño limitado del vocabulario y hay una falta de backup sobre los caracteres.

In [6]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(trainDataset.map(lambda text, label: text))
vocab = np.array(encoder.get_vocabulary())
print(vocab[:20])

['' '[UNK]' 'the' 'and' 'a' 'of' 'to' 'is' 'in' 'it' 'i' 'this' 'that'
 'br' 'was' 'as' 'for' 'with' 'movie' 'but']


Mostrando ejemplos del encoder

In [7]:
for example, label in trainDataset.take(1):
    encodedExample = encoder(example)[:3].numpy()
    print(encodedExample)

for example, label in trainDataset.take(1):
    encodedExample = encoder(example)[:3].numpy()
    for n in range(3):
        print('Original: ', example[n].numpy())
        print('Round-trip: ', ' '.join(vocab[encodedExample[n]]))
        print()

[[  1  11 729 ...   0   0   0]
 [297   1  88 ...   0   0   0]
 [  1  14   4 ...   0   0   0]]
Original:  b'Everyone is either loving or hating this film. I am going with loving. It is so well shot and so well acted. Beautiful. This film is for people who appreciate well crafted film making. If you are not a fan of well done films of course you would hate this. But if you like the tops of acting, photography, story and development, look no further then here.'
Round-trip:  everyone is either [UNK] or [UNK] this film i am going with [UNK] it is so well shot and so well [UNK] beautiful this film is for people who [UNK] well [UNK] film making if you are not a fan of well done films of course you would hate this but if you like the [UNK] of acting [UNK] story and development look no [UNK] then here                                                                                                                                                                                                     

Implementando arquitectura para RNN

In [8]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1),
])

print([layer.supports_masking for layer in model.layers])


[False, True, True, True, True]


Viendo si funciona la solución

Texto sin padding

In [9]:
sampleText = ('The movie was cool. The animation and the graphics '
              'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sampleText]))
predictions[0]

array([-0.00460901], dtype=float32)

Texto con padding

In [10]:
padding = "the " * 2000
predictions = model.predict(np.array([sampleText, padding]))
predictions[0]

array([-0.00460901], dtype=float32)

Compilando la solución

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy']
              )
model.summary()

Entrenando modelo

In [ ]:
history = model.fit(trainDataset, epochs=10,
                    validation_data=testDataset, validation_steps=30)

testLoss, testAcc = model.evaluate(testDataset)
print('Test Loss: ', testLoss)
print('Test Accuracy: ', testAcc)

# Resultados

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plotGraphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plotGraphs(history, 'loss')
plt.ylim(0, None)

![](graphs.png)

Una de las ventajas de usar Tensorflow es exportar este modelo para no cargar nuevamente el entrenamiento y gastar tiempo. Así, para poder usarlo existen varias formas, pero dado que mi objetivo es usarlo en una plataforma web, utilice Flask. Este framework de python permite hacer una comunicación TCP/IP de forma fácil y sencilla. Cree una directiva para que Flask obtuviera el texto a analizar y el modelo se encarga de predicir qué tan bueno o malo es, como si lo estuviera usando en local.

De esta forma, la implementación para el módulo de comentarios queda de la siguiente forma.

![Home](Home.jpg)

Después de escribir texto, sale el siguiente resultado.

![Comment](Comment.jpg)